### Exercises - Indicative Solutions - Discussion

In [20]:
# Some auxiliary functions

# Reading a fast file
def readfasta(fastafile):
    import regex as re
    f=open(fastafile, 'r')
    seq = ""
    total = 0
    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    f.close()
    return(seq)

# A maximum function
def maximum(x, y) :
    if x > y :
        return x
    else :
        return y


# 1. Palindromes. Finding them, extracting the greatest

In [57]:
# Take 1: Iteration 

def longestpalindrome(s):
  palindromes=[]
  for i in range(1,len(s)):
    try:
        m=1 
        while s[i-m]==s[i+m]:
            palindromes.append(s[i-m:i+m+1])
            m+=1
    except:
        pass
  return max(palindromes, key=len)

# Run
myseq=readfasta("../files/ecoli.fa")
myseq=ecoliseq.replace('N','')
myseq='AATGVGVVVGCCCGXATAAAATAXX'
longestpalindrome(myseq)

'GVVVG'

The solution above only works for odd-sized palindromes. 
What about even-sized ones?

In [52]:
# Take 2: Recursion

# we will build on a simple recursive function like the one given below

def palindrome(mystring):
    if len(mystring) < 2:
        return True
    if mystring[0] != mystring[-1]:
        return False
    return palindrome(mystring[1:-1])


# We will next scan the sequence of the genome and call the function 

def recursivePalindrome(s):
    palindromes = []
    for i in range(1,len(s)):
        k = 0
        subsequence = s[i]
        if (palindrome(subsequence)):
            print(subsequence)
            palindromes.append(subsequence)
            k += 1
            subsequence = s[i-k:i+k]
        else:
            next            
            
    return max(palindromes, key=len)

recursivePalindrome("AATGTVV")

A
T
G
T
V
V


'A'

# 2. Non-mers in E.coli Genome

Below I provide code taken from one of your assignments, slightly modified and with comments.

In [ ]:
with open ("../files/ecoli.fa") as l:
    l.readline()
    ecolseq =""
    for ln in l:
        lnseq = ln.strip()
        ecolseq += lnseq

#Findind all the possible 10-mers that can be found with the 4 nucleotides
import itertools

k=10
bases = ["A","T","G","C"]
combos = itertools.product(bases, repeat=k)
strcomb = ("".join(y) for y in combos) #generators save memory compared to lists

#Find all k-mers/10-mers in the genome

kmers=[]
for i in range(len(ecolseq)-k+1):
    kmers.append(ecolseq[i:i+k])

#Isolation of all the unique ones (makes the procedure faster)
kmers = set(kmers)

#Calculating how many 10-mers are not included in the E.coli genome
count=0
nonkmers =[]
allk =0
res =0
for y in strcomb:
    allk += 1     #control to check if the theoritical 4^10 has been produced with the product functional tool 
    if y not in kmers:
        count +=1 #counting the 10-mers not included is same as len(nonkemers)/not necessary / check point
        nonkmers.append(y)
    else:         #not necessary / check point
        res+=1    #counting the included 10-mers / check point


import random

print("The number of non 10-mers is:",count)
print("Some of those non-mers are", random.sample(nonkmers,10))

# POINT: We can do the same without looping, simply by comparing sets
allkmers = set([''.join(p) for p in itertools.product(bases, repeat=k)])
nonmers = allkmers.difference(kmers)
print("Found:", len(nonmers), "nonmers in the genome of E.coli")

#Expected non-10-mers/ Theoritical check: 
theor = 4**10
obs = len(kmers)
exp = theor - obs
print("According to theory we would expect", exp,"non-mers since all the 10-mers are", theor,"and the observed in E.Coli are", obs)

# NOTE: The theoritical check is not in agreement with the ones found above with comparison. Not quite sure why.

# =================================================================================================================
# POINT: This is not correct from a theoretical viewpoint. In fact it is much more complex. Can you think of why?
# =================================================================================================================


#Another way to avoid for-loops and exhausting search is to immediately produce sets that contain unique 10-mers and calculate their difference:
#The reading of the file is common

# -> This is the point made above! (OK)

kmers2 = {ecolseq[i:i+k] for i in range(len(ecolseq)-k+1) }
combos = itertools.product(bases, repeat=k) 
#must to re-write the command since it's a generator that has been used above and needs to be generated to reused
strcomb2 = {"".join(y) for y in combos} #permuation with replacement we can make it a set as they all uniq
nonmers = strcomb2 - kmers2

print(len(nonmers))
print(random.sample(nonmers, 10)) #demonstartion of some non-mers

Found: 223321  in the genome of E.coli
The number of non 10-mers is: 223321
Some of those non-mers are ['TCACTTACCA', 'ACAGCTGTTA', 'ATGAGGCTAG', 'TTTCGTGATT', 'GCCTCGGGCG', 'ACTAACTTCC', 'CATTCTATTA', 'TCGATTCAAG', 'ACATATTAAG', 'GCCTATGGTA']
According to theory we would expect 223302 non-mers since all the 10-mers are 1048576 and the observed in E.Coli are 825274
223321
['TAGGGTCACG', 'GTACACGAGG', 'CACCGTATAG', 'TCGTTTAGAG', 'TTTAGTCGCG', 'GATCTCATGT', 'AAGGACTGTT', 'GCCCCACCGA', 'CATCTCACAC', 'AGAATGCTAA']


Question: Did anyone try to use the binary search? Would it be faster?

# 3. Gibbs Sampler

Below I am attaching an indicative solution

In [48]:
# Gibbs Sampling to locate a motif in a set of sequences
# code
sequences = []
with open ('../files/motifs_in_sequence.fa') as file:
    for line in file:
        sequences.append(line.strip()) # 50 sequences as elements of a list. 100 bases each sequence
    

##Gibbs sampler##
import random
import numpy as np

def Gibbs_sampler(sequences,k): #k is the length of the motif, sequence is a list with the sequences
    
    dictionary = {'A':0,
                  'T':1,
                  'C':2,
                  'G':3}
    
    column_sum = len(sequences) #number of rows (50) or number of sequences
    length = len(sequences[0]) #number of columns or number of nucleotides in seq
    Imax = 1.8*k #threshold of I
    
    pwm = np.zeros([4,k]) # A,T,C,G X len(motif)
    
    for seq in sequences:
        rand_start = random.randint(0, length-k) #pick a random nucleotide from each sequence
        motif = seq[rand_start:rand_start+k] #and take substring as the motif
       
        lst = enumerate(motif) #finding the index of each nucleotide in the motif to access the correct column
                               #and using the dictionary to access the correct row 
        for i in lst: #making the first random pwm
            pwm[dictionary[i[1]],i[0]]+=1
            
    pwm = pwm/column_sum
    
    information = np.zeros([1,k])
    count=0
    while (np.sum(information)) < Imax: #while information_content of the pwm 
        motives=[]                      #is less than the threshold
        
        information_old = np.sum(information) #keeping the previous value of information contect
                                             #to check convergence in case the theshold 
        for row in range (column_sum):                #is never reached
            maxx=0
            rand_seq = random.randint(0, column_sum-1) #pick a random index - sequence
            seq = sequences[rand_seq]               
            for i in range(len(seq)-k):   #take each k-mer from the sequence 
                score = 0
                substring = seq[i:i+k]
                lst = enumerate(substring)
                
                for j in lst:                         #scoring each k-mer based on the pwm
                    score+=pwm[dictionary[j[1]],j[0]]   #keeping the motif with the highest score
                                                         #from each sequence
                if score > maxx:  
                    maxx = score
                    motif = substring
                    
            motives += [motif] #keep all the motifs with the highest score in the list motives
        
        pwm = np.zeros([4,k]) # A,T,C,G X len(motif) 
        
        for elem in motives: 
            lst = enumerate(elem)
            for i in lst:         #making the new pwm
                pwm[dictionary[i[1]],i[0]]+=1
                 
        pwm = pwm/column_sum
        
        information = np.zeros([1,k]) #computing the information of each position
        for i in range(k):
            information[0,i] = 2-abs(sum([elem*np.log2(elem) for elem in pwm[:,i] if elem > 0]))
            

        if abs(information_old - np.sum(information)) <= 0.5: #ckecking convergence 
            count+=1
            if count == 10: #if the difference of the information content is less or equal to 0.5
                break       #for consecutive 10 iterations then break
        else:
            count=0
    
    max_index_col = np.argmax(pwm, axis=0) #extracting the motif according to the   
                                           #highest frequency of each nucleotide in each position
    motif=''
    for values in max_index_col:
        for keys in dictionary.keys():
            if values == dictionary[keys]:
                motif+= keys
        
    return pwm,information,motif


#repeat the algorithmm 100 times for each k (3 to 7) and keep the pwm and motif with the highest infromation_content
#this process takes approximately 4min (in my computer)
    
####100-cycled GIbbs#####
for k in range (3,8):
    highest_info = 0
    for i in range (100):
        summ=0
        pwm, information_content,motif = Gibbs_sampler(sequences,k)
        summ+=np.sum(information_content)
        if summ > highest_info:
            highest_info = summ
            pwm_ret = pwm
            motif_ret = motif
        
    print('\nThe information content of the motif divided by it\'s length is:',highest_info/k) #divide by length to normalize and compare among other k
    print('The pwm of the motif is:\n',pwm_ret)
    print('The motif is:',motif_ret)



#To find the motifs for each k with the highest information content i have to repeat gibbs sampler many times because of the randomness that takes place
#The motif that returns the highest scaled information content is GAT (k = 3, I/k = 1.9528 or 2!) but i think that the motif that we are looking for
#is the motif GATA (k=4, I/k = 1.857) which contains GAT and is contained in all the longer found motifs.
#(Also we know the existence of the GATA transcription factors and indeed GATA is part of these binding sites)
#Sometimes the algorithm returns other 3-mers (eg ATG,GGC,AAG) with high information content as well but it
#is noted that GATA is returned almost in every repetition of the 100-cycled Gibbs which means that it may be the only 4-mer motif with so high information content.
#Finally i have to report that the threshold that i chose for checking convergence may not be the best choice and should be stricter
#Either way i kept this threshold because otherwise i would have made the algorithm a lot slower (is already slow though :P).


The information content of the motif divided by it's length is: 2.0
The pwm of the motif is:
 [[0. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
The motif is: CGT

The information content of the motif divided by it's length is: 1.8640950668139766
The pwm of the motif is:
 [[0.   1.   0.   0.92]
 [0.02 0.   1.   0.08]
 [0.   0.   0.   0.  ]
 [0.98 0.   0.   0.  ]]
The motif is: GATA

The information content of the motif divided by it's length is: 1.5973762912220875
The pwm of the motif is:
 [[0.6  0.   1.   0.   0.94]
 [0.24 0.   0.   0.98 0.  ]
 [0.12 0.   0.   0.   0.02]
 [0.04 1.   0.   0.02 0.04]]
The motif is: AGATA

The information content of the motif divided by it's length is: 1.3910716042455649
The pwm of the motif is:
 [[0.48 0.   1.   0.   0.88 0.44]
 [0.46 0.02 0.   1.   0.12 0.16]
 [0.02 0.   0.   0.   0.   0.04]
 [0.04 0.98 0.   0.   0.   0.36]]
The motif is: AGATAA

The information content of the motif divided by it's length is: 1.1870405050107355
The pwm of the motif is:
